In [14]:
from scipy.sparse import csr_matrix,rand,coo_matrix
import tables as tb
import numpy as np
import pandas as pd
from numpy import array
import time

In [113]:
file=pd.read_table("/Users/jma7/Development/hail/test/genotypeChr22.tsv",index_col=0)

In [114]:
sparsefile=file.to_sparse(fill_value=0)

In [2]:
def sparse_df_to_array(df):
    num_rows = df.shape[0]   

    data = []
    row = []
    col = []

    for i, col_name in enumerate(df.columns):
        if isinstance(df[col_name], pd.SparseSeries):
            column_index = df[col_name].sp_index
            if isinstance(column_index, pd._sparse.BlockIndex):
                column_index = column_index.to_int_index()

            ix = column_index.indices
            data.append(df[col_name].sp_values)
            row.append(ix)
            col.append(len(df[col_name].sp_values) * [i])
        else:
            data.append(df[col_name].values)
            row.append(np.array(range(0, num_rows)))
            col.append(np.array(num_rows * [i]))

    data_f = np.concatenate(data)
    row_f = np.concatenate(row)
    col_f = np.concatenate(col)

    arr = coo_matrix((data_f, (row_f, col_f)), df.shape, dtype=np.float64)
    return arr.tocsr()

In [8]:
mat=csr_matrix(file.values)

In [36]:
mat=csr_matrix(file.as_matrix())

In [36]:
def store_sparse_dataframe(data, name, store='storeChr22.h5'):
    m=csr_matrix(data.as_matrix())
    rownames=data.index.values
    colnames=data.columns.values
    msg = "This code only works for csr matrices"
    assert(m.__class__ == csr_matrix), msg
    filters = tb.Filters(complevel=9, complib='blosc')
    with tb.open_file(store,'a') as f:
        for par in ('data', 'indices', 'indptr', 'shape'):
            full_name = '%s_%s' % (name, par)
            try:
                n = getattr(f.root, full_name)
                n._f_remove()
            except AttributeError:
                pass

            arr = array(getattr(m, par))
            atom = tb.Atom.from_dtype(arr.dtype)
            ds = f.create_carray(f.root, full_name, atom, arr.shape,filters=filters)
            print(f)
            ds[:] = arr
        ds = f.create_carray(f.root, name+"_rownames",  tb.StringAtom(itemsize=200), rownames.shape,filters=filters)
        print(f)
        ds[:] = rownames
        ds = f.create_carray(f.root, name+"_colnames", tb.StringAtom(itemsize=100), colnames.shape,filters=filters)
        print(f)
        ds[:] = colnames
        
def binarySearchName(nameArray,name):
    start=0
    end=len(nameArray)-1
    found=False
    mid=0
    while (start<=end and not found):
        mid=(end+start)//2
        if (name==nameArray[mid]):
            found=True
        else:
            if (name<nameArray[mid]):
                end=mid-1
            elif (name>nameArray[mid]):
                start=mid+1
    if not found:
        mid=-1
    return mid
    
        
        
def load_genotype_by_variantID_HDF5(name,ids, store='store.h5'):
    start_time = time.time()
    with tb.open_file(store) as f:
        pars = []
        for par in ('data', 'indices', 'indptr', 'shape',"rownames","colnames"):
            pars.append(getattr(f.root, '%s_%s' % (name, par)).read())
    m = csr_matrix(tuple(pars[:3]), shape=pars[3])
    rownames=pars[4].astype('str')
    colnames=pars[5].astype('str')
    ids=rownames[:10000]
    print("--- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for id in ids:
#         valindex=np.nonzero(rownames==id)[0][0]
        valindex=binarySearchName(rownames,id)
        value=m.getrow(valindex)
        cols=value.indices
        genotypes=value.data
        for index,pos in enumerate(cols):
            sampleName=colnames[pos]
            value=genotypes[index]
#             print(id,sampleName,value)
    print("--- %s seconds ---" % (time.time() - start_time))
    return

def load_genotype_by_ids_samples_HDF5(name,ids,samples, store='store.h5'):
    with tb.open_file(store) as f:
        pars = []
        for par in ('data', 'indices', 'indptr', 'shape',"rownames","colnames"):
            pars.append(getattr(f.root, '%s_%s' % (name, par)).read())
    m = csr_matrix(tuple(pars[:3]), shape=pars[3])
    rownames=pars[4].astype('str')
    colnames=pars[5].astype('str')
    ids=rownames[:10000]
    samples=colnames[:20]
    genotypes=m.data
    start_time = time.time()
    for id in ids:
        idIndex=binarySearchName(rownames,id)
        if idIndex!=-1:
            value=m.getrow(idIndex)
            cols=value.indices
            genotypes=value.data
#             print(cols)
            for sample in samples:
                sampleIndex=binarySearchName(colnames,sample)
                if sampleIndex!=-1:
                    samplePos=binarySearchName(cols,sampleIndex)
                    if samplePos!=-1:
                        genotype=genotypes[samplePos]
#                         print(id,sample,genotype)
    print("--- %s seconds ---" % (time.time() - start_time))
    return

def load_sparse_dataframe(name, store='store.h5'):
    with tb.open_file(store) as f:
        pars = []
        for par in ('data', 'indices', 'indptr', 'shape',"rownames","colnames"):
            pars.append(getattr(f.root, '%s_%s' % (name, par)).read())
    m = csr_matrix(tuple(pars[:3]), shape=pars[3])
    return m
        

In [ ]:
store_sparse_dataframe(file,"dataTable")

In [13]:
def store_sparse_mat(m, name, store='store.h5'):
    msg = "This code only works for csr matrices"
    assert(m.__class__ == csr_matrix), msg
    filters = tb.Filters(complevel=9, complib='blosc')
    with tb.open_file(store,'a') as f:
        for par in ('data', 'indices', 'indptr', 'shape'):
            full_name = '%s_%s' % (name, par)
            try:
                n = getattr(f.root, full_name)
                n._f_remove()
            except AttributeError:
                pass

            arr = array(getattr(m, par))
            atom = tb.Atom.from_dtype(arr.dtype)
            ds = f.create_carray(f.root, full_name, atom, arr.shape,filters=filters)
            print(f)
            ds[:] = arr

def load_sparse_mat(name, store='store.h5'):
    with tb.open_file(store) as f:
        pars = []
        for par in ('data', 'indices', 'indptr', 'shape'):
            pars.append(getattr(f.root, '%s_%s' % (name, par)).read())
    m = csr_matrix(tuple(pars[:3]), shape=pars[3])
    return m

In [4]:
ids=['chr22:g.16052080G>A',"chr22:g.16063484G>A","chr22:g.16202725C>A","chr22:g.16235504_16235507del"]
samples=['HG00096',"HG00102","HG00728","HG01577"]

In [38]:
load_genotype_by_ids_samples_HDF5("dataTable",ids,samples,store='store.h5')

--- 7.722084999084473 seconds ---


In [116]:
file.shape

(1096485, 2504)

In [21]:
%time load_genotype_by_variantID_HDF5("dataTable",['chr22:g.16052080G>A',"chr22:g.16063484G>A","chr22:g.16202725C>A","chr22:g.16235504_16235507del"],store='storeChr22.h5')

--- 4.073753118515015 seconds ---
--- 2.465273141860962 seconds ---
CPU times: user 4.45 s, sys: 1.36 s, total: 5.81 s
Wall time: 6.72 s


In [39]:
matcheck=load_sparse_mat("table")

In [40]:
filecheck=matcheck.toarray()

In [ ]:
loadDataFrame=load_sparse_dataframe("table")


In [17]:
type(filecheck)

numpy.ndarray

In [52]:
matcheck.indptr[:20]

array([  0,   1,  33,  70,  71,  72,  74,  79,  81,  82,  83,  85,  86,
        87,  88, 125, 126, 164, 190, 196], dtype=int32)